In [1]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB

iris = load_iris()
df_X = pd.DataFrame(iris.data, columns = iris.feature_names)
df_Y = pd.DataFrame(iris.target, columns = ["target"])

X_train, test_X, y_train, test_y = train_test_split(df_X, df_Y, train_size=0.8, test_size=0.2, random_state=123)

In [2]:
X_train.tail()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
17,5.1,3.5,1.4,0.3
98,5.1,2.5,3.0,1.1
66,5.6,3.0,4.5,1.5
126,6.2,2.8,4.8,1.8
109,7.2,3.6,6.1,2.5


In [3]:
gnb = GaussianNB()

fitted = gnb.fit(X_train, y_train)
y_pred = fitted.predict(test_X)
y_pred
print(y_pred)

print("테스트 데이터 수 : %d, 틀린 개수 : %d" 
      % (test_X.shape[0], (np.array(test_y.target.tolist()) != y_pred).sum()))

[1 2 2 1 0 2 1 0 0 1 2 0 1 2 2 2 0 0 1 0 0 1 0 2 0 0 0 2 2 0]
테스트 데이터 수 : 30, 틀린 개수 : 1


C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [4]:
print(fitted.classes_)
print(fitted.class_count_)
print(test_X[:1],"\n")
print(fitted.predict_proba(test_X)[:1],"\n")
print(fitted.predict(test_X)[:1],"\n")

[0 1 2]
[37. 44. 39.]
    sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)
72                6.3               2.5                4.9               1.5 

[[7.24143720e-126 9.23061979e-001 7.69380215e-002]] 

[1] 



In [5]:
print(fitted.predict_proba(test_X)[[1,20]])
fitted.predict(test_X)[[1,20]]

[[1.81805757e-197 1.22131524e-005 9.99987787e-001]
 [1.00000000e+000 8.57846082e-017 1.77097412e-025]]


array([2, 0])

In [6]:
from sklearn.metrics import confusion_matrix
confusion_matrix(test_y, y_pred)

array([[13,  0,  0],
       [ 0,  6,  0],
       [ 0,  1, 10]], dtype=int64)

In [7]:
test_X.head(1)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
72,6.3,2.5,4.9,1.5


In [8]:
predict_data = np.array(test_X.iloc[0])
predict_data

array([6.3, 2.5, 4.9, 1.5])

In [9]:
fitted.theta_

array([[5.01621622, 3.43243243, 1.46756757, 0.25945946],
       [5.95      , 2.78409091, 4.24090909, 1.32272727],
       [6.58717949, 2.95897436, 5.57948718, 2.02820513]])

In [10]:
fitted.sigma_

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:103: FutureWarning: Attribute `sigma_` was deprecated in 1.0 and will be removed in1.2. Use `var_` instead.
  warnings.warn(msg, category=FutureWarning)


array([[0.10568298, 0.14975895, 0.02705625, 0.01214025],
       [0.27068182, 0.10042872, 0.22741736, 0.04221075],
       [0.39752795, 0.11011177, 0.29188692, 0.0774096 ]])

In [11]:
fitted.class_prior_

array([0.30833333, 0.36666667, 0.325     ])

In [12]:
likelihood = [
# 0 클래스
(sp.stats.norm(fitted.theta_[0][0], np.sqrt(fitted.sigma_[0][0])).pdf(predict_data[0]) * \
sp.stats.norm(fitted.theta_[0][1], np.sqrt(fitted.sigma_[0][1])).pdf(predict_data[1]) * \
sp.stats.norm(fitted.theta_[0][2], np.sqrt(fitted.sigma_[0][2])).pdf(predict_data[2]) * \
sp.stats.norm(fitted.theta_[0][3], np.sqrt(fitted.sigma_[0][3])).pdf(predict_data[3])),\
# 1 클래스
(sp.stats.norm(fitted.theta_[1][0], np.sqrt(fitted.sigma_[1][0])).pdf(predict_data[0]) * \
sp.stats.norm(fitted.theta_[1][1], np.sqrt(fitted.sigma_[1][1])).pdf(predict_data[1]) * \
sp.stats.norm(fitted.theta_[1][2], np.sqrt(fitted.sigma_[1][2])).pdf(predict_data[2]) * \
sp.stats.norm(fitted.theta_[1][3], np.sqrt(fitted.sigma_[1][3])).pdf(predict_data[3])),\
# 2 클래스
(sp.stats.norm(fitted.theta_[2][0], np.sqrt(fitted.sigma_[0][0])).pdf(predict_data[0]) * \
sp.stats.norm(fitted.theta_[2][1], np.sqrt(fitted.sigma_[0][1])).pdf(predict_data[1]) * \
sp.stats.norm(fitted.theta_[2][2], np.sqrt(fitted.sigma_[0][2])).pdf(predict_data[2]) * \
sp.stats.norm(fitted.theta_[2][3], np.sqrt(fitted.sigma_[0][3])).pdf(predict_data[3])) 
]
likelihood


[2.0700298536453225e-126, 0.2218869448618605, 7.497361843154609e-09]

In [14]:
posterior = likelihood * fitted.class_prior_
print(posterior)
posterior/np.sum(posterior, axis=0)

[6.38259205e-127 8.13585464e-002 2.43664260e-009]


array([7.84501707e-126, 9.99999970e-001, 2.99494353e-008])

In [15]:
print(fitted.predict_proba(test_X)[[0]])

[[7.24143720e-126 9.23061979e-001 7.69380215e-002]]


In [16]:
#사전확률설정
gnb2 = GaussianNB(priors = [1/100, 1/100, 98/100])
fitted2 = gnb2.fit(iris.data, iris.target)
y_pred2 = fitted2.predict(iris.data)
confusion_matrix(iris.target,y_pred2)

array([[50,  0,  0],
       [ 0, 33, 17],
       [ 0,  0, 50]], dtype=int64)

In [17]:
gnb3 = GaussianNB(priors = [1/100, 98/100, 1/100])
fitted3 = gnb3.fit(iris.data, iris.target)
y_pred3 = fitted3.predict(iris.data)
confusion_matrix(iris.target,y_pred3)

array([[50,  0,  0],
       [ 0, 50,  0],
       [ 0, 14, 36]], dtype=int64)

In [18]:
#다항분포 나이브 베이즈
from sklearn.naive_bayes import MultinomialNB
import numpy as np 

mNB_train_X = np.random.randint(4,size=(4,10))
mNB_train_y = np.array([1,2,3,4])

In [20]:
mNB_train_X

array([[2, 3, 1, 3, 3, 0, 0, 1, 1, 1],
       [0, 1, 2, 3, 3, 3, 1, 1, 2, 1],
       [2, 1, 2, 0, 3, 1, 3, 3, 2, 0],
       [2, 2, 0, 0, 3, 0, 2, 3, 3, 2]])

In [22]:
mNB_train_y

array([1, 2, 3, 4])

In [23]:
# 학습
mNB = MultinomialNB()
fitted_mNB = mNB.fit(mNB_train_X, mNB_train_y)
# 첫 번째 값의 예측 결과(클래스) 확인
print(fitted_mNB.predict(mNB_train_X[:1]))
# 첫 번째 값의 각 클래스별 확률 확인
fitted_mNB.predict_proba(mNB_train_X[:1])

[1]


array([[0.98282931, 0.00968215, 0.00136155, 0.00612699]])

In [24]:
clmNB_2 = MultinomialNB(class_prior = [0.1,0.5,0.1,0.1])
clmNB_2.fit(mNB_train_X,mNB_train_y)

MultinomialNB(class_prior=[0.1, 0.5, 0.1, 0.1])

In [25]:
clmNB_2.predict_proba(mNB_train_X[:1]) 

array([[0.9461849 , 0.04660577, 0.00131079, 0.00589854]])